In [10]:
%reload_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

from dmsensei import DataModule, create_model
from dmsensei.config import device
from dmsensei.core.callbacks import WandbFitLogger, KaggleLogger, WandbTestLogger
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch import Trainer
from dmsensei.config import device
import sys
import os
from lightning.pytorch.loggers import WandbLogger
import torch, wandb
import numpy as np
torch.manual_seed(0)
np.random.seed(0)

device = 'cuda'

# Create dataset
dm = DataModule(
    name=["ribo-kaggle"],
    data_type=["dms", "shape",'structure'],
    force_download=False,
    batch_size=32,
    num_workers=0,
    train_split=256,
    valid_split=0,
    predict_split=0,
    overfit_mode=True,
    shuffle_valid=False,
)

dm.setup('fit')

Loading dataset from disk...


In [40]:

import torch
import torch.nn.functional as F

lr = 1E-3
model = create_model(
    model="transformer",
    # weight_data=True,
    ntoken=5,
    d_model=16,
    nhead=16,
    d_hid=13,
    nlayers=2,
    dropout=0.0,
    lr=lr,
    weight_decay=0,
    gamma=0.997,
).to(device)

/root/DMSensei/dmsensei/models/transformer.py:108: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight * 0.001)


In [41]:
wandb.init(project="dmsensei", name="small_tranfo-mseloss-lr:{}".format(lr))

trainer = Trainer(
    max_epochs=1000,
    log_every_n_steps=1,
    callbacks=[
        WandbFitLogger(dm=dm, batch_size=dm.batch_size, log_plots_every_n_epoch=100000),
    ],
    logger=WandbLogger(),
)

trainer.fit(model, dm)

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/loss,▇█▇▇▇▇▇▆▆▆▆▇▅▆▆▅▅▄▅▅▄▅▄▄▃▄▃▄▄▃▂▂▂▂▁▂▂▂▁▂
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,49
train/loss,0.27619
trainer/global_step,1599


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/root/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type               | Params
-----------------------------------------------------------
0 | loss_signal_fn      | MSELoss            | 0     
1 | encoder             | Embedding          | 80    
2 | pos_encoder         | PositionalEncoding | 0     
3 | transformer_encoder | Sequential         | 3.2 K 
4 | resnet              | Sequential         | 9.7 K 
5 | output_net_DMS      | Sequential         | 1.2 K 
6 | output_net_SHAPE    | Sequential         | 1.2 K 
7 | output_net          | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/root/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.
/root/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/utilities/data.py:104: Total length of `DataLoader` across ranks is zero. Please make sure this was your intention.
/root/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=27` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

/root/.pyenv/versions/3.11.6/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
%reload_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
import json

data = json.load(open('/root/DMSensei/data/datafolders/ribo-kaggle/data.json'))


In [ ]:
mini_data = {}
for idx, (key, val) in enumerate(data.items()):
    if idx > 100:
        break
    mini_data[key] = val

In [ ]:

from dmsensei import DataModule, create_model
from dmsensei.config import device
from dmsensei.core.callbacks import WandbFitLogger, KaggleLogger, WandbTestLogger
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch import Trainer
from dmsensei.config import device
import sys
import os
from lightning.pytorch.loggers import WandbLogger
import torch, wandb
import numpy as np
torch.manual_seed(0)
np.random.seed(0)
from torch.utils.data import Dataset as TorchDataset
from typing import List
from dmsensei.core.batch import Batch
from dmsensei.huggingface import get_dataset
from dmsensei.config import device
from dmsensei.core.datatype import DMSDataset, SHAPEDataset, StructureDataset
import torch.nn.functional as F
from dmsensei.core.embeddings import sequence_to_int
from dmsensei.core.util import _pad

class Dataset(TorchDataset):
    def __init__(
        self,
        name: str,
        data_type: List[str],
        force_download: bool,
        tqdm=True,
    ) -> None:
        super().__init__()
        self.name = name
        self.data_type = data_type
        data = get_dataset(
            name=name,
            force_download=force_download,
            tqdm=tqdm,
        )
        self.length = [len(d['sequence']) for d in data.values()]
        self.refs = list(data.keys())
        self.L = max(self.length)
        self.sequence = torch.stack([_pad(sequence_to_int(data[ref]['sequence']), self.L, 'sequence') for ref in self.refs])
        self.dms, self.shape, self.structure = None, None, None
        if 'dms' in data_type:
            self.dms = DMSDataset.from_data_json(data, self.L, self.refs)
        if 'shape' in data_type:
            self.shape = SHAPEDataset.from_data_json(data, self.L, self.refs)
        if 'structure' in data_type:
            self.structure = StructureDataset.from_data_json(data, self.L, self.refs)

    def __len__(self) -> int:
        return len(self.sequence)

    def __getitem__(self, index) -> tuple:
        return {
            'reference': self.refs[index],
            'sequence': self.sequence[index],
            'length': self.length[index],
            'dms': self.dms[index] if self.dms is not None else None,
            'shape': self.shape[index] if self.shape is not None else None,
            'structure': self.structure[index] if self.structure is not None else None,
        }

    def collate_fn(self, batch_data):
        batch = Batch.from_dataset_items(batch_data, self.data_type)
        return batch

            
dataset = Dataset(
    name = 'test',
    data_type = ['dms', 'shape', 'structure'],
    force_download=0
)


In [ ]:
dataset[5]

In [ ]:
from dmsensei.core.datatype import DataType
from dmsensei.core.datatype import data_type_factory
import torch

def split_data_type(data_type):
    if not "_" in data_type:
        data_part = "true"
    else:
        data_part, data_type = data_type.split("_")
    return data_part, data_type


class Batch:
    def __init__(
        self,
        reference,
        sequence,
        length,
        L,
        batch_size,
        data_types,
        dms=None,
        shape=None,
        structure=None,
    ):
        self.reference = reference
        self.sequence = sequence
        self.length = length
        self.dms = dms
        self.shape = shape
        self.structure = structure
        self.L = L
        self.batch_size = batch_size
        self.data_types = data_types

    @classmethod
    def from_dataset_items(cls, datapoints: list, data_type: str):
        reference= [d['reference'] for d in datapoints]
        length = [d['length'] for d in datapoints]
        L = max(length)
        sequence= torch.stack([d['sequence'][:L] for d in datapoints])
        batch_size = len(datapoints)
        
        data = {}
        for dt in data_type:
            true, error, index = [], [], []
            for idx, dp in enumerate(datapoints):
                if dt in dp and dp[dt] is not None:
                    true.append(dp[dt]['true'])
                    if dt != 'structure':
                        error.append(dp[dt]['error']) 
                    index.append(idx)
            
            if len(true) == 0:
                continue
            

            if dt != 'structure':
                data[dt] = data_type_factory['batch'][dt](
                    true=torch.stack(true),
                    pred=None,
                    error=torch.stack(error),
                    index=torch.tensor(index),
                )
            else:
                data[dt] = data_type_factory['batch'][dt](
                    true=torch.stack(true),
                    pred=None,
                    index=torch.tensor(index),
                )
            
        return cls(
            reference=reference,
            sequence=sequence,
            length=length,
            L=L,
            batch_size=batch_size,
            data_types=data_type,
            **data,
        )
        

    def get(self, data_type, index=None, to_numpy=False):
        if not self.contains(data_type):
            raise ValueError(f"Batch does not contain {data_type}")
        
        if data_type in ["reference", "sequence", "length"]:
            out = getattr(self, data_type)
        else:
            data_part, data_type = split_data_type(data_type)
            out = getattr(getattr(self, data_type), data_part)
        
        if index is not None:
            out = out[index]
            if hasattr(out, '__len__'):
                out = out[:self.get("length")[index]]

        if to_numpy:
            if hasattr(out, "cpu"):
                out = out.squeeze().cpu().numpy()
        return out
        
batch = dataset.collate_fn([dataset[0], dataset[1], dataset[2]])

In [ ]:
batch.get('sequence')

In [ ]:
%reload_ext autoreload
%autoreload 2
from dmsensei.core.dataset import Dataset
from dmsensei.core.batch import Batch

dataset = Dataset(
    name = 'test',
    data_type = ['dms', 'shape', 'structure'],
    force_download=0
)

In [ ]:
b = Batch.from_dataset_items([dataset[0], dataset[1], dataset[2]], ['dms', 'shape', 'structure'])

In [ ]:
b.get('error_shape')